# Solver for a multifocal Koehler integrator

In [1]:
import itertools

import numpy as np
from scipy.optimize import minimize

from kmdouglass.udesigner import Units
from kmdouglass.udesigner.mfki import compute_results, DEFAULTS

## Define the input search space

In this case, we'll loop over (microlens array) MLA focal lengths and pitches.

In [2]:
mla_f = np.array([4.88, 10.0])
mla_pitch = np.array([300.0, 600.0])

search_space = list(itertools.product(mla_f, mla_pitch))

## Define the cost function

In [3]:
def cost_function(
        mla_f: float,
        mla_pitch: float,
        mla_f_units: Units = Units.mm,
        mla_pitch_units: Units = Units.um,
        field_size_target: float = 160e-6,
        field_size_hyperparam: float = 1.0,
        spot_size_hyperparam: float = 1.0,
        fresnel_number_hyperparam: float = 1.0,
        homogeneity_hyperparam: float = 1.0,
) -> float:
    inputs = DEFAULTS.copy()
    inputs["mla.focal_length"] = mla_f
    inputs["mla.focal_length.units"] = mla_f_units
    inputs["mla.pitch"] = mla_pitch
    inputs["mla.pitch.units"] = mla_pitch_units

    results = compute_results(inputs)

    flat_field_size_sample_plane = results["flat_field_size_sample_plane"]["value"] * results["flat_field_size_sample_plane"]["units"].value
    excitation_spot_size_sample_plane = results["excitation_spot_size_sample_plane"]["value"] * results["excitation_spot_size_sample_plane"]["units"].value
    fresnel_number = results["fresnel_number"]["value"]
    homogeneity = results["homogeneity"]["value"]

    size_err = np.abs(flat_field_size_sample_plane - field_size_target)**2

    cost = (
        field_size_hyperparam * size_err
        + spot_size_hyperparam * excitation_spot_size_sample_plane
        + fresnel_number_hyperparam * fresnel_number
        + homogeneity_hyperparam * homogeneity
    )

    return cost

## Run the optimization routine

In [4]:
def objective(params):
    mla_f, mla_pitch = params
    return cost_function(mla_f, mla_pitch)

In [6]:
result = minimize(
    objective,
    search_space[0],
    bounds=[(mla_f.min(), mla_f.max()), (mla_pitch.min(), mla_pitch.max())],
    method="Nelder-Mead",
)

(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)


In [7]:
result

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 25.733728492349847
             x: [ 1.000e+01  4.092e+02]
           nit: 42
          nfev: 78
 final_simplex: (array([[ 1.000e+01,  4.092e+02],
                       [ 1.000e+01,  4.092e+02],
                       [ 1.000e+01,  4.092e+02]]), array([ 2.573e+01,  2.573e+01,  2.573e+01]))

In [ ]:
search_space

[(4.88, 300.0), (4.88, 600.0), (10.0, 300.0), (10.0, 600.0)]